In [1]:
import os
import pandas as pd
import numpy as np

print("Imported Modules")

Imported Modules


In [2]:
'''
SET THE WORKING DIRECTORY BELOW TO THE LOCATION OF DATA FILES
'''
working_directory = 'C:/PythonWD' #set location using back slashes

os.chdir(working_directory)

print("Current working directory: {0}".format(os.getcwd()))


def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            output_path = os.makedirs(directory)
            print(output_path)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        

# Folder where outputs will be saved (by default a folder within the working directory) 
createFolder('./output/') 
output_path = 'C:/PythonWD/output/'

print('Import Modules: Done')

Current working directory: C:\PythonWD
Import Modules: Done


In [3]:
'''
Carbon premium and risk factors

Replication of Table 10: Can the carbon premium be explained by risk factors?

a 1 ,t = c 0 + c F t + ε t ,

Time series variables:

α = monthly carbon premium estimated in cross-sectional returns regression

MKTRF is the monthly return on the value-weighted stock market net of the risk-free rate
HML is the monthly return on the portfolio long value stocks and short growth stocks
SMB is the monthly return on the portfolio long small-cap stocks and short, large-cap stocks
MOM is the monthly return on the portfolio long 12-month stock winners and short 12-month past losers
CMA is the monthly return of a portfolio that is long on conservative investment stocks and short on aggressive investment stocks
BAB is the monthly return of a portfolio that is long on low-beta stocks and short on high-beta stocks
LIQ is the liquidity factor of Pastor and Stambaugh
NET ISSUANCE is the monthly return of a portfolio that is long on high-net-issuance stocks and short on low-net-issuance stocks. Net issuance for year t is the change in the natural log of split-adjusted shares outstanding from the fiscal yearend in t -2 to the fiscal yearend in t -1
IDIO VOL is the monthly return of a portfolio that is long on low idiosyncratic volatility stocks and short on high idiosyncratic volatility stocks.

   
'''

'\nCarbon premium and risk factors\n\nReplication of Table 10: Can the carbon premium be explained by risk factors?\n\na 1 ,t = c 0 + c F t + ε t ,\n\nTime series variables:\n\nα = monthly carbon premium estimated in cross-sectional returns regression\n\nMKTRF is the monthly return on the value-weighted stock market net of the risk-free rate\nHML is the monthly return on the portfolio long value stocks and short growth stocks\nSMB is the monthly return on the portfolio long small-cap stocks and short, large-cap stocks\nMOM is the monthly return on the portfolio long 12-month stock winners and short 12-month past losers\nCMA is the monthly return of a portfolio that is long on conservative investment stocks and short on aggressive investment stocks\nBAB is the monthly return of a portfolio that is long on low-beta stocks and short on high-beta stocks\nLIQ is the liquidity factor of Pastor and Stambaugh\nNET ISSUANCE is the monthly return of a portfolio that is long on high-net-issuance 

In [4]:
"MKTRF"

#import zero coupon 0 year (1 month) RBA yield data (risk free rate)
#SOURCE RBA, Mnemonic = FZCY0D, ZERO-COUPON INTEREST RATES (YIELD)
rf = pd.read_csv ('rba_1m_yield.csv', encoding='latin1')

#import ASX ALL ORDS price data
rm = pd.read_csv ('asx_all_ords.csv', encoding='latin1')


#get monthly yields
rf['date'] = pd.to_datetime(rf['date'], format='%d-%b-%Y')
rf = rf.set_index(['date'])
rf = rf.resample('M').mean()
rf = rf.loc['2008-07':'2022-06'] #sample period 
rf = rf.reset_index()
rf['year'] = rf['date'].dt.strftime('%Y')
rf['month'] = rf['date'].dt.strftime('%m')

#get monthly ASX_ORDS returns
rm['date'] = pd.to_datetime(rm['date'], format='%d/%m/%Y')
rm = rm.set_index(['date'])
rm = rm.resample('M').ffill()
rm['price_lag'] = rm['ASX ALL ORDINARIES - PRICE INDEX'].shift(1)
rm['rm'] = (rm['ASX ALL ORDINARIES - PRICE INDEX']/rm['price_lag']) - 1
rm = rm.loc['2008-07':'2022-06']
rm = rm.reset_index()
rm['year'] = rm['date'].dt.strftime('%Y')
rm['month'] = rm['date'].dt.strftime('%m')


mktrf = pd.merge (rf[['year', 'month','zero_coupon_yield_0yrs']], rm[['year', 'month','rm']], on = ['year','month'], how = 'inner')
mktrf['zero_coupon_yield_0yrs'] = mktrf['zero_coupon_yield_0yrs'] / 100
mktrf['mktrf'] = mktrf['rm'] - mktrf['zero_coupon_yield_0yrs']

In [5]:
"HML"

'''
BE/ME - The portfolio HML (high minus low). meant to mimic the risk
factor in returns related to book-to-market equity, is defined similarly. HML is
the difference, each month, between the simple average of the returns on the two
high-BE/ME portfolios (S/H and B/H) and the average of the returns on the two
low- BE/ME portfolios (S/L and B/L). The two components of HML are returns
on high- and low-BE/ME portfolios with about the same weighted-average size.
Thus the difference between the two returns should be largely free of the size
factor in returns, focusing instead on the different return behaviors of high- and
low-BE/ME firms.

Get S/H, B/H, S/L, B/L
'''

'\nBE/ME - The portfolio HML (high minus low). meant to mimic the risk\nfactor in returns related to book-to-market equity, is defined similarly. HML is\nthe difference, each month, between the simple average of the returns on the two\nhigh-BE/ME portfolios (S/H and B/H) and the average of the returns on the two\nlow- BE/ME portfolios (S/L and B/L). The two components of HML are returns\non high- and low-BE/ME portfolios with about the same weighted-average size.\nThus the difference between the two returns should be largely free of the size\nfactor in returns, focusing instead on the different return behaviors of high- and\nlow-BE/ME firms.\n\nGet S/H, B/H, S/L, B/L\n'